In [2]:
# import some function we can use later
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import sys
from IPython.display import display, Image
from six.moves import cPickle as pickle
%matplotlib inline


In [3]:
import time

In [4]:
#load data first
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS+NWP.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset= save['t_v_dataset']
    test_dataset = save['test_dataset']
    del save

In [5]:
train_time = train_dataset['time']
train_data = np.hstack((train_dataset['data_ASOS'], train_dataset['data_NWP']))
train_label = train_dataset['label']

test_time = test_dataset['time']
test_data = np.hstack((test_dataset['data_ASOS'], test_dataset['data_NWP']))
test_label = test_dataset['label']

In [6]:
#dataset normalize
mean = train_data.mean(axis = 0)
std = train_data.std(axis = 0)
print(mean.shape, std.shape)
train_data_n = (train_data - mean)/std
test_old_data_n = (test_data - mean)/std

(82,) (82,)


In [7]:
train_label[:100]

array([  7.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
         8.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.])

In [8]:
from sklearn.preprocessing import Binarizer

In [10]:
pre = Binarizer(threshold = 1.01)
b_train_label = pre.transform(train_label.reshape(1, -1))
b_test_old_label = pre.transform(test_label.reshape(1, -1))

In [11]:
b_train_label[:, :30][0].shape

(30,)

In [12]:
c1 = b_train_label.sum(axis = 1)[0] 

In [13]:
c2 = b_train_label.shape[1] - c1

In [14]:
print(c1, c2, c2*100/(c1+c2))

114677.0 1980.0 1.6972834892


In [15]:
#need to balanced the dataset
from unbalanced_dataset.over_sampling import SMOTE

In [16]:
c_train_label = 1 - b_train_label[0]
c_test_old_label = 1 - b_test_old_label[0]

In [18]:
sm = SMOTE(ratio = 0.03, kind='regular')
train_data_n_resample, c_train_label_resample = sm.fit_transform(train_data_n, c_train_label)

Determining classes statistics... 2 classes detected: Counter({0.0: 114677, 1.0: 1980})
Finding the 5 nearest neighbours...
done!
Creating synthetic samples...Generated 1460 new samples ...
done!


In [19]:
test_old_data_n_resample, c_test_old_label_resample = sm.fit_transform(test_old_data_n, c_test_old_label)

Determining classes statistics... 2 classes detected: Counter({0.0: 28744, 1.0: 421})
Finding the 5 nearest neighbours...
done!
Creating synthetic samples...Generated 441 new samples ...
done!


In [20]:
train_data_n_resample.shape, c_train_label_resample.shape

((118117, 82), (118117,))

In [21]:
c1 = c_train_label_resample.sum()
c2 = c_train_label_resample.shape[0] - c1
print(c1, c2, c1*100/(c1+c2))

3440.0 114677.0 2.91236655181


In [18]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

clf.fit(train_data_n, c_train_label)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
predict_c = clf.predict(test_old_data_n)
TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

TP=245, FP=72, FN=477, CSI=0.309, POD=0.339, FAR=0.227


In [28]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
s_t = time.time()
clf.fit(train_data_n, c_train_label)
e_t = time.time() 

In [29]:

predict_c = clf.predict(test_old_data_n)
TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))
print(e_t - s_t)

TP=208, FP=66, FN=213, CSI=0.427, POD=0.494, FAR=0.241
28.2735159397


In [46]:
predict_c = clf.predict(test_old_data_n)
TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

TP=641, FP=2657, FN=81, CSI=0.190, POD=0.888, FAR=0.806


In [30]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier()
s_t = time.time()
clf.fit(train_data_n, c_train_label)
e_t = time.time() 

In [31]:

predict_c = clf.predict(test_old_data_n)
TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))
print(e_t - s_t)

TP=243, FP=156, FN=178, CSI=0.421, POD=0.577, FAR=0.391
0.443342924118


In [40]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier()
clf.fit(train_data_n_resample, c_train_label_resample)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [41]:
predict_c = clf.predict(test_old_data_n_resample)
TP = ((predict_c + c_test_old_label_resample)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label_resample.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

TP=46467, FP=3552, FN=3379, CSI=0.870, POD=0.932, FAR=0.071


In [44]:
predict_c = clf.predict(test_old_data_n)
print(predict_c.shape)
TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % ( TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

(50568,)
TP=667, FP=3552, FN=55, CSI=0.156, POD=0.924, FAR=0.842


In [35]:
from sklearn.tree import DecisionTreeClassifier
for max_d in range(2, 11):
    clf = DecisionTreeClassifier(max_depth = max_d)
    s_t = time.time()
    clf.fit(train_data_n_resample, c_train_label_resample)
    e_t = time.time()
    predict_c = clf.predict(test_old_data_n)
    TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
    FP = predict_c.sum() - TP
    FN = c_test_old_label.sum() - TP
    print("depth=%d, TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (max_d, TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))
    print(e_t - s_t)

depth=2, TP=279, FP=146, FN=142, CSI=0.492, POD=0.663, FAR=0.344
2.26705312729
depth=3, TP=252, FP=92, FN=169, CSI=0.491, POD=0.599, FAR=0.267
3.47003602982
depth=4, TP=252, FP=92, FN=169, CSI=0.491, POD=0.599, FAR=0.267
4.21723008156
depth=5, TP=256, FP=112, FN=165, CSI=0.480, POD=0.608, FAR=0.304
5.40498805046
depth=6, TP=261, FP=141, FN=160, CSI=0.464, POD=0.620, FAR=0.351
6.24978113174
depth=7, TP=254, FP=133, FN=167, CSI=0.458, POD=0.603, FAR=0.344
7.5666179657
depth=8, TP=247, FP=155, FN=174, CSI=0.429, POD=0.587, FAR=0.386
7.96973514557
depth=9, TP=259, FP=177, FN=162, CSI=0.433, POD=0.615, FAR=0.406
9.20372700691
depth=10, TP=262, FP=202, FN=159, CSI=0.421, POD=0.622, FAR=0.435
9.80379295349


In [48]:
from sklearn.tree import DecisionTreeClassifier
for max_d in range(2, 11):
    clf = DecisionTreeClassifier(max_depth = max_d)
    clf.fit(train_data_n_resample, c_train_label_resample)
    predict_c = clf.predict(test_old_data_n)
    TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
    FP = predict_c.sum() - TP
    FN = c_test_old_label.sum() - TP
    print("depth=%d, TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (max_d, TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

depth=2, TP=633, FP=1794, FN=89, CSI=0.252, POD=0.877, FAR=0.739
depth=3, TP=647, FP=2101, FN=75, CSI=0.229, POD=0.896, FAR=0.765
depth=4, TP=646, FP=2443, FN=76, CSI=0.204, POD=0.895, FAR=0.791
depth=5, TP=628, FP=2725, FN=94, CSI=0.182, POD=0.870, FAR=0.813
depth=6, TP=625, FP=2010, FN=97, CSI=0.229, POD=0.866, FAR=0.763
depth=7, TP=613, FP=1876, FN=109, CSI=0.236, POD=0.849, FAR=0.754
depth=8, TP=602, FP=1486, FN=120, CSI=0.273, POD=0.834, FAR=0.712
depth=9, TP=574, FP=1339, FN=148, CSI=0.279, POD=0.795, FAR=0.700
depth=10, TP=566, FP=1136, FN=156, CSI=0.305, POD=0.784, FAR=0.667


In [36]:
from sklearn.ensemble import RandomForestClassifier
for max_d in range(2, 21):
    clf = RandomForestClassifier(max_depth = max_d)
    s_t = time.time()
    clf.fit(train_data_n, c_train_label)
    e_t = time.time()
    predict_c = clf.predict(test_old_data_n)
    TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
    FP = predict_c.sum() - TP
    FN = c_test_old_label.sum() - TP
    print("depth=%d, TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (max_d, TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))
    print(e_t - s_t)

depth=2, TP=103, FP=3, FN=318, CSI=0.243, POD=0.245, FAR=0.028
1.80295991898
depth=3, TP=161, FP=25, FN=260, CSI=0.361, POD=0.382, FAR=0.134
2.63570785522
depth=4, TP=173, FP=27, FN=248, CSI=0.386, POD=0.411, FAR=0.135
3.29041600227
depth=5, TP=182, FP=40, FN=239, CSI=0.395, POD=0.432, FAR=0.180
4.05445504189
depth=6, TP=213, FP=54, FN=208, CSI=0.448, POD=0.506, FAR=0.202
4.76007890701
depth=7, TP=188, FP=52, FN=233, CSI=0.397, POD=0.447, FAR=0.217
5.35415482521
depth=8, TP=188, FP=52, FN=233, CSI=0.397, POD=0.447, FAR=0.217
6.06815886497
depth=9, TP=202, FP=62, FN=219, CSI=0.418, POD=0.480, FAR=0.235
6.6959400177
depth=10, TP=166, FP=56, FN=255, CSI=0.348, POD=0.394, FAR=0.252
7.54036808014
depth=11, TP=168, FP=58, FN=253, CSI=0.351, POD=0.399, FAR=0.257
7.53257203102
depth=12, TP=192, FP=68, FN=229, CSI=0.393, POD=0.456, FAR=0.262
8.11895990372
depth=13, TP=179, FP=54, FN=242, CSI=0.377, POD=0.425, FAR=0.232
8.8984310627
depth=14, TP=186, FP=74, FN=235, CSI=0.376, POD=0.442, FAR=0.28

In [37]:
from sklearn import svm
clf = svm.SVC(kernel='linear', C=1.0)
s_t = time.time()
clf.fit(train_data_n, c_train_label)
e_t = time.time()

In [38]:
predict_c = clf.predict(test_old_data_n)
TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label.sum() - TP
print(" TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % ( TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))
print(e_t - s_t)

 TP=207, FP=63, FN=214, CSI=0.428, POD=0.492, FAR=0.233
821.065064907
